# <font color='#F37126'> ETL Final </font>

### Imports 

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import requests

from datetime import datetime
from time     import sleep

from bs4 import BeautifulSoup
from selenium.webdriver                import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

from sqlalchemy import create_engine
import re

### Data Collection

In [2]:
# parameters
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# URL
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

# Request to URL
page = requests.get( url, headers=headers )

# Beatiful Sou object
soup = BeautifulSoup( page.text, 'html.parser' ) 

# ================== Product Data =================
products = soup.find( 'ul', class_='products-listing small')
product_list = products.find_all( 'article', class_='hm-product-item')

# product id
product_id = [p.get( 'data-articlecode' ) for p in product_list]

# product_category
product_category = [p.get( 'data-category' ) for p in product_list]

# product name
product_list = products.find_all( 'a', class_='link' )
product_name = [p.get_text() for p in product_list]

# price
product_list = products.find_all( 'span', class_='price regular')
product_price = [p.get_text() for p in product_list]

data = pd.DataFrame( [product_id, product_category, product_name,
                  product_price] ).T
data.columns = ['product_id', 'product_category', 'product_name',
                  'product_price']

In [4]:
data.shape

(36, 4)

### Data collection by products

In [8]:
# chrome driver options
chrome_options = Options()  
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--allow-running-insecure-content')
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')
chrome_options.add_argument("--headless") 

# instantiate chrome driver 
browser = Chrome(options=chrome_options)
print('Passou do browser')

# empty dataframe
df_compositions = pd.DataFrame()

# unique columns for all products
aux = []

cols = ['Art. No.', 'Composition', 'Fit', 'More sustainable materials', 'Product safety', 'Size']
df_pattern = pd.DataFrame( columns=cols )

cont = 0
for i in range( len( data ) ):
    cont += 1
    print(cont, end='-')
    # API Requests
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i, 'product_id'] + '.html'
    page = requests.get( url, headers=headers )  
    
    # Beautiful Soup object
    soup = BeautifulSoup( page.text, 'html.parser' )
 
    product_list = soup.find_all( 'a', class_='filter-option miniature active' ) + soup.find_all( 'a', class_='filter-option miniature' ) 
   
    #   color name
    color_name = []
    
    #   product id
    product_id = []
    
    for p in product_list:

        url2 = 'https://www2.hm.com' + p.get( 'href' )

        browser.get( url2 )
        print('pegou a url2')
        sleep(0.5)

        text_ = browser.find_element(By.XPATH, "//*[@id='picker-1']/button/span[1]").text
        print(f'O texto encontrado foi: {text_}')
        sleep(0.5)
        
        if (text_ == 'Select size'):   

            color_name.append( p.get( 'data-color' ) )
            product_id.append( p.get( 'data-articlecode' ) )
            

    df_color = pd.DataFrame( [product_id, color_name] ).T
    df_color.columns = ['product_id', 'color_name']
    
    for j in range( len( df_color ) ):
        # API Requests
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j, 'product_id'] + '.html'

        page = requests.get( url, headers=headers )  
    
        # Beautiful Soup object
        soup = BeautifulSoup( page.text, 'html.parser' )
        
        # Product name
        product_name = soup.find_all( 'h1', class_='primary product-item-headline' )
        product_name = product_name[0].get_text()
        
        # Product price
        product_price = soup.find_all( 'div', class_='primary-row product-item-price')
        product_price = re.findall( r'\d+\.?\d+', product_price[0].get_text() )[0]
        
        # composition
        product_composition_list = soup.find_all( 'div', class_='pdp-description-list-item' )
        product_composition = [list( filter( None, p.get_text().split( '\n' ) ) ) for p in product_composition_list]

        df_composition = pd.DataFrame( product_composition ).T
        df_composition.columns = df_composition.iloc[0]

        # delete first row
        df_composition = df_composition.iloc[1:].fillna( method='ffill' )

        # remove pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Pocket lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Pocket: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Shell: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace( 'Lining: ', '', regex=True)


        # garantee the same number of columns
        df_composition = pd.concat( [df_pattern, df_composition], axis=0)    

        # rename columns
        df_composition.columns = ['product_id', 'composition', 'fit', 'more_sustainable_materials', 'product_safety', 'size']
        df_composition['product_name'] = product_name
        df_composition['price'] = product_price
        
        # keep new columns if it show up
        aux = aux + df_composition.columns.tolist()

        # merge data color + composition
        df_composition = pd.merge( df_composition, df_color, how='left', on='product_id')

        # all products
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0)
        
# Join Showroom data + details      
df_compositions['style_id'] = df_compositions['product_id'].apply( lambda x: x[:-3])
df_compositions['color_id'] = df_compositions['product_id'].apply( lambda x: x[-3:])

# scrapy datetime
df_compositions['scrapy_datetime'] = datetime.now().strftime( '%Y-%m-%d %H:%M:%S' )

Passou do browser
1-pegou a url2
O texto encontrado foi: Select size
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Select size
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Select size
pegou a url2
O texto encontrado foi: Out of stock
pegou a url2
O texto encontrado foi: Select size
pegou a url2
O texto encontrado foi: Select size
2-pegou a url2
O texto encontrado foi: Select size


KeyboardInterrupt: 

In [8]:
df_compositions.head()

,product_id,composition,fit,more_sustainable_materials,product_safety,size,product_name,price,color_name,style_id,color_id,scrapy_datetime
0,0690449043,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Light denim blue/trashed,0690449,043,2021-12-28 08:41:38
1,0690449043,"Polyester 65%, Cotton 35%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Light denim blue/trashed,0690449,043,2021-12-28 08:41:38
0,0690449022,Polyester 100%,Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black/trashed,0690449,022,2021-12-28 08:41:38
1,0690449022,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black/trashed,0690449,022,2021-12-28 08:41:38
0,0690449036,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 188cm/6'2"" and wears a size 31/30",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black,0690449,036,2021-12-28 08:41:38


In [9]:
len( df_compositions['product_id'].unique() )

57

In [14]:
df_compositions.head()

,product_id,composition,fit,more_sustainable_materials,product_safety,size,product_name,price,color_name,style_id,color_id,scrapy_datetime
0,0690449043,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Light denim blue/trashed,0690449,043,2021-12-28 08:41:38
1,0690449043,"Polyester 65%, Cotton 35%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Light denim blue/trashed,0690449,043,2021-12-28 08:41:38
0,0690449022,Polyester 100%,Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black/trashed,0690449,022,2021-12-28 08:41:38
1,0690449022,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 187cm/6'2"" and wears a size 32/32",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black/trashed,0690449,022,2021-12-28 08:41:38
0,0690449036,"Cotton 98%, Spandex 2%",Skinny fit,NaN,NaN,"The model is 188cm/6'2"" and wears a size 31/30",\n\t\t\t\t\t\t\t Skinny Jeans,39.99,Black,0690449,036,2021-12-28 08:41:38


### Data Cleaning

In [10]:
# product id
df_data =  df_compositions.dropna( subset=['product_id'] )

# product name
df_data['product_name'] = df_data['product_name'].str.replace('\n', '')
df_data['product_name'] = df_data['product_name'].str.replace('\t', '')
df_data['product_name'] = df_data['product_name'].str.replace('  ', '')
df_data['product_name'] = df_data['product_name'].str.replace(' ', '_').str.lower()

# # product price
df_data['price'] = df_data['price'].astype( float )

# color name
df_data['color_name'] = df_data['color_name'].str.replace( ' ', '_' ).str.lower()

# fit
df_data['fit'] = df_data['fit'].apply( lambda x: x.replace( ' ', '_').lower() if pd.notnull( x ) else x )

# size number
df_data['size_number'] = df_data['size'].apply( lambda x: re.search( '\d{3}cm', x ).group(0) if pd.notnull( x ) else x )
df_data['size_number'] = df_data['size_number'].apply( lambda x: re.search( '\d+', x ).group(0) if pd.notnull( x ) else x)

# size model
df_data['size_model'] = df_data['size'].str.extract( '(\d+/\\d+)' )

# break composition by comma
df1 = df_data['composition'].str.split( ',', expand=True ).reset_index( drop=True )


# cotton | polyester | spandex | elasterel
df_ref = pd.DataFrame( index=np.arange( len( df_data ) ), columns=['cotton', 'polyester', 'spandex', 'elasterell'])

# =====================composition=======================

# --------------cotton--------------
df_cotton_0 = df1.loc[df1[0].str.contains( 'Cotton', na=True ), 0]
df_cotton_0.name = 'cotton'

df_cotton_1 = df1.loc[df1[1].str.contains( 'Cotton', na=True ), 1]
df_cotton_1.name = 'cotton'

# combine
df_cotton = df_cotton_0.combine_first( df_cotton_1 )

df_ref = pd.concat( [df_ref, df_cotton], axis=1 )
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]

# ---------------polyester--------------
df_polyester_0 = df1.loc[df1[0].str.contains( 'Polyester', na=True ), 0]
df_polyester_0.name = 'polyester'

df_polyester_1 = df1.loc[df1[1].str.contains( 'Polyester', na=True ), 1]
df_polyester_1.name = 'polyester'

df_polyester = df_polyester_0.combine_first( df_polyester_1 )

df_ref = pd.concat( [df_ref, df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]

# ------------------spandex------------------
df_spandex_1 = df1.loc[df1[1].str.contains( 'Spandex', na=True ), 1]
df_spandex_1.name = 'spandex'

df_spandex_2 = df1.loc[df1[2].str.contains( 'Spandex', na=True ), 2]
df_spandex_2.name = 'spandex'

# combina spandex from both columns 1 and 2
df_spandex = df_spandex_1.combine_first( df_spandex_2 )

df_ref = pd.concat( [df_ref, df_spandex], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]

# -----------------elasterell-----------------------
df_elasterell = df1.loc[df1[1].str.contains( 'Elasterell', na=True ), 1]
df_elasterell.name = 'elasterell'

df_ref = pd.concat( [df_ref, df_elasterell], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated( keep='last' )]


# join of combine with product_id
df_aux = pd.concat( [df_data['product_id'].reset_index(drop=True), df_ref], axis=1 )

# format composition data
df_aux['cotton'] = df_aux['cotton'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['polyester'] = df_aux['polyester'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['spandex'] = df_aux['spandex'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )
df_aux['elasterell'] = df_aux['elasterell'].apply( lambda x: int( re.search( '\d+', x ).group(0) ) / 100 if pd.notnull( x ) else x )

# final join
df_aux = df_aux.groupby( 'product_id' ).max().reset_index().fillna(0)
df_data = pd.merge( df_data, df_aux, on='product_id', how='left' )


# Drop columns
df_data = df_data.drop( columns=['size', 'product_safety', 'composition', 'more_sustainable_materials'], axis=1)

# Drop duplicate
df_data = df_data.drop_duplicates()
df_data.shape

(57, 14)

In [11]:
df_data.head(11)

,product_id,fit,product_name,price,color_name,style_id,color_id,scrapy_datetime,size_number,size_model,cotton,polyester,spandex,elasterell
0,0690449043,skinny_fit,skinny_jeans,39.99,light_denim_blue/trashed,0690449,043,2021-12-28 08:41:38,187,32/32,0.98,0.65,0.02,0.0
2,0690449022,skinny_fit,skinny_jeans,39.99,black/trashed,0690449,022,2021-12-28 08:41:38,187,32/32,0.98,1.00,0.02,0.0
4,0690449036,skinny_fit,skinny_jeans,39.99,black,0690449,036,2021-12-28 08:41:38,188,31/30,0.98,0.00,0.02,0.0
5,0690449051,skinny_fit,skinny_jeans,39.99,gray,0690449,051,2021-12-28 08:41:38,NaN,NaN,0.98,0.00,0.02,0.0
6,0690449056,skinny_fit,skinny_jeans,39.99,light_denim_blue,0690449,056,2021-12-28 08:41:38,NaN,NaN,0.98,0.65,0.02,0.0
8,0985159001,skinny_fit,skinny_jeans,19.99,black,0985159,001,2021-12-28 08:41:38,187,32/32,1.00,0.00,0.01,0.0
10,0985159002,skinny_fit,skinny_jeans,19.99,denim_blue,0985159,002,2021-12-28 08:41:38,187,31/32,1.00,0.00,0.01,0.0
12,0985159003,skinny_fit,skinny_jeans,19.99,dark_gray,0985159,003,2021-12-28 08:41:38,NaN,NaN,0.99,0.63,0.01,0.0
14,0985159004,skinny_fit,skinny_jeans,19.99,light_denim_blue,0985159,004,2021-12-28 08:41:38,NaN,NaN,1.00,0.00,0.01,0.0
16,0985159005,skinny_fit,skinny_jeans,19.99,dark_blue,0985159,005,2021-12-28 08:41:38,189,32/32,1.00,0.00,0.01,0.0


# Data Insert

In [23]:
data_insert = df_data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color_name',
    'fit',
    'price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'elasterell',
    'scrapy_datetime'
]]

In [24]:
data_insert.head()

,product_id,style_id,color_id,product_name,color_name,fit,price,size_number,size_model,cotton,polyester,spandex,elasterell,scrapy_datetime
0,0690449043,0690449,043,skinny_jeans,light_denim_blue/trashed,skinny_fit,39.99,187,32/32,0.98,0.65,0.02,0.0,2021-12-28 08:41:38
2,0690449022,0690449,022,skinny_jeans,black/trashed,skinny_fit,39.99,187,32/32,0.98,1.00,0.02,0.0,2021-12-28 08:41:38
4,0690449036,0690449,036,skinny_jeans,black,skinny_fit,39.99,188,31/30,0.98,0.00,0.02,0.0,2021-12-28 08:41:38
5,0690449051,0690449,051,skinny_jeans,gray,skinny_fit,39.99,NaN,NaN,0.98,0.00,0.02,0.0,2021-12-28 08:41:38
6,0690449056,0690449,056,skinny_jeans,light_denim_blue,skinny_fit,39.99,NaN,NaN,0.98,0.65,0.02,0.0,2021-12-28 08:41:38


In [25]:
query_showroom_schema = """
    CREATE TABLE vitrine(
        product_id      TEXT,
        style_id        TEXT,
        color_id        TEXT,
        product_name    TEXT,
        color_name      TEXT,
        fit             TEXT,
        price           REAL,
        size_number     TEXT,
        size_model      TEXT,
        cotton          REAL,
        polyester       REAL,
        spandex         REAL,
        elasterell      REAL,
        scrapy_datetime TEXT
        
    )
"""


In [19]:
# query_drop = """
# DROP TABLE vitrine
# """

In [26]:
# create table
conn = sqlite3.connect( 'database_hm.sqlite' )
# cursor = conn.execute( query_drop )
cursor = conn.execute( query_showroom_schema )
conn.commit()

In [21]:
# create database connection
conn = create_engine( 'sqlite:///database_hm.sqlite', echo=False)

# data insert
data_insert.to_sql( 'vitrine', con=conn, if_exists='append', index=False )

In [22]:
!pwd

/home/judson/Downloads/python-ds-ao-dev/modulo-9
